In [1]:
import getpass
import os
from langchain_core.prompts import ChatPromptTemplate
from langchain_experimental.tools import PythonAstREPLTool
from langchain.output_parsers.openai_tools import JsonOutputKeyToolsParser
import pandas as pd
from io import StringIO
from typing import Annotated
from typing_extensions import TypedDict

from langchain_core.messages import ToolMessage, AIMessage
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from pandas.core.frame import DataFrame
from pandas.core.series import Series
import json
import importlib

# os.environ["OPENAI_API_KEY"] = getpass.getpass('Password: ')
api_key = "sk-pzmVDXe1-rxgtV34fEob57Owp9b7Z7_k-jYKmFaguDT3BlbkFJXR835WsPGSV7P_sjJXS2QGOJBIxgW5wSzwqExklvAA"

from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini", api_key=api_key)

In [2]:
def create_function_generation_chain(csv_file_path_list, LLM_model, function_name):
    df_dict = {}
    df_info_list = []
    buffer = StringIO()
    cur = 1
    for file_path in csv_file_path_list:
        df = pd.read_csv(file_path, delimiter=',')
        file_name = file_path.split('/')[-1]
        print(f"Read the file {file_name}!")
        df.info(buf=buffer)
        info_lines = buffer.getvalue().splitlines()
        trimmed_info = '\n'.join(info_lines[1:-2])
        df_name = f"csv_{cur}"
        cur += 1
        df_info_list.append((df_name, trimmed_info))
        buffer.truncate(0)
        buffer.seek(0)
        df_dict[df_name] = df.copy()
        print(f"{df_name} : {file_path}")

    df_template = """```
    {df_name}: 
    >>> {df_info}
    ```"""

    df_context = "\n\n".join(
        df_template.format(df_name=df_name, df_info=df_info)
        for df_name, df_info in df_info_list
    )

    tool = PythonAstREPLTool(locals=df_dict)
    llm_with_tools = LLM_model.bind_tools([tool], tool_choice=tool.name)
    parser = JsonOutputKeyToolsParser(key_name=tool.name, first_tool_only=True)
    df_context = df_context.replace("{", "{{").replace("}", "}}")
    # Modify the system message with clearer instructions if necessary
    system = f"""You have access to several pandas DataFrames. The following information contains the structure (column names and types) for each DataFrame:
    {df_context}

    Based on a user's request about the data, generate Python code that uses only built-in Python libraries, pandas. The code must be valid, complete, and functional.

    Here are the key requirements:
    1. Include all necessary import statements at the beginning of the code.
    2. Ensure the code handles potential column name or index errors by carefully checking the DataFrame's structure.
    3. Apply any necessary data preprocessing steps to handle missing data, incorrect data types, or any other common issues that could arise in the data. Pay special attention to ensuring that the data types are correct and consistent across columns to avoid type-related errors. Unify the data type if necessary
    4. Ensure the intermediate state dataframe is stored in memory, if necessary, for future processing steps.
    5. Return only the Python code—no explanations, comments, or extra text.
    6. Wrap all the operations into a function called '{function_name}' whose input is a string called '社員番号' and a dict called df_dict same as dataframe name and dataframe passed above, and output is a dataframe and return it.'"""

    prompt = ChatPromptTemplate.from_messages(
        [("system", system), ("human", "{question}")])

    chain_until_parser = (prompt | llm_with_tools | parser)  # noqa
    return chain_until_parser, df_dict


In [11]:
file_path_1 = '/Users/formaideveloper/LLM_excel/テスト用データ/other_1/calc_data.csv'
chain, df_dict = create_function_generation_chain([file_path_1], llm, "function_1")

Read the file calc_data.csv!
csv_1 : /Users/formaideveloper/LLM_excel/テスト用データ/other_1/calc_data.csv


In [12]:
prompt = "'社員番号'が与えられている場合、次のように列を見つける：部門、就業規則、出勤日数、総出勤時間数。"
res = chain.invoke(prompt)

In [13]:
with open("./function_1.py", 'w') as f:
    f.write(res["query"])

In [14]:
func = getattr(importlib.import_module("function_1"), "function_1")
print(func)

<function function_1 at 0x7fa68cccf520>


In [15]:
print(func("854430", df_dict))

             部門        就業規則区分  出勤日数  勤務時間合計
0  エグゼクティブ　オフィス  正社員_管理監督者_本社   0.0     NaN


In [3]:
func = getattr(importlib.import_module("generated_functions.function_1"), "function_1")